# Implementing Gates

This function shows how to implement various gates in TensorFlow.

One gate will be one operation with a variable and the input tensor of our model. We will ask TensorFlow 
to change the variable based on our loss function!

In [ ]:
import tensorflow as tf

### Gate 1

Create a multiplication gate:  $f(x) = a * x$
```
  a --
      |
      |---- (multiply) --> output
      |
  x --
```

In [ ]:
# Initialize variables and input data
a = tf.Variable(4.)
x_data = tf.keras.Input(shape=(1,))
x_val = 5.


# Add a layer which computes f(x) = a * x
multiply_layer = tf.keras.layers.Lambda(lambda x:tf.multiply(a, x))
outputs = multiply_layer(x_data)

# Build the model
model = tf.keras.Model(inputs=x_data, outputs=outputs, name="gate_1")

print(model.summary())

# Optimizer
optimizer=tf.keras.optimizers.SGD(0.01)


# Run loop across gate
print('Optimizing a Multiplication Gate Output to 50.')
for i in range(10):
    
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        
        # Forward pass.
        mult_output = model(x_val)
        
        # Loss value as the difference between
        # the output and a target value, 50.
        loss_value = tf.square(tf.subtract(mult_output, 50.))
        
    # Get gradients of loss with reference to the variable "a" to adjust.
    gradients = tape.gradient(loss_value, a)
    
    # Update the variable "a" of the model.
    optimizer.apply_gradients(zip([gradients], [a]))
    
    print("Step: {} ==> {} * {} = {}".format(i, a.numpy(), x_val, a.numpy() * x_val))

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=() dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "gate_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None
Optimizing a Multiplication Gate Output to 50.
Step: 0 ==> 7.0 * 5.0 = 35.0
Step: 1 ==> 8.5 * 5.0 = 42.5
Step: 2 ==> 9.25 * 5.0 = 46.25
Step: 3 ==> 9.625 * 5.0 = 48

In [ ]:
# Instead of using a lambda layer, we can also use a subclassed layer
class MyCustomMultiplyLayer(tf.keras.layers.Layer):
 
 def __init__(self, units):
   super(MyCustomMultiplyLayer, self).__init__()
   self.units = units
   self.a = tf.Variable(4.)

 def call(self, inputs):
   return inputs * self.a


# Initialize variables
x_data = tf.keras.Input(dtype=tf.float32, shape=(1,))
a = tf.Variable(4, dtype=tf.float32)

# Add a layer which computes f(x) = a * x
multiply_layer = MyCustomMultiplyLayer(units=1)
outputs = multiply_layer(x_data)

# Build the model
model = tf.keras.Model(inputs=x_data, outputs=outputs, name="gate_1")

#print(model.summary())

# Optimizer
optimizer=tf.keras.optimizers.SGD(0.01)

# Run loop across gate
print('Optimizing a Multiplication Gate Output to 50.')
for i in range(10):
    
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        
        # Forward pass.
        mult_output = model(5.)
        
        # Loss value as the difference between
        # the output and a target value, 50.
        loss_value = tf.square(tf.subtract(mult_output, 50.))
    
    # Get gradients of loss with reference to the variable "a" to adjust.
    gradients = tape.gradient(loss_value, multiply_layer.a)
    
    # Update the weights of the model.
    optimizer.apply_gradients(zip([gradients], [multiply_layer.a]))
    
    print("Step: {} ==> {} * {} = {}".format(i, multiply_layer.a.numpy(), x_val, multiply_layer.a.numpy() * x_val))

Optimizing a Multiplication Gate Output to 50.
Step: 0 ==> 7.0 * 5.0 = 35.0
Step: 1 ==> 8.5 * 5.0 = 42.5
Step: 2 ==> 9.25 * 5.0 = 46.25
Step: 3 ==> 9.625 * 5.0 = 48.125
Step: 4 ==> 9.8125 * 5.0 = 49.0625
Step: 5 ==> 9.90625 * 5.0 = 49.53125
Step: 6 ==> 9.953125 * 5.0 = 49.765625
Step: 7 ==> 9.9765625 * 5.0 = 49.8828125
Step: 8 ==> 9.98828125 * 5.0 = 49.94140625
Step: 9 ==> 9.994140625 * 5.0 = 49.970703125


### Gate 2

Create a nested gate: $f(x) = a * x + b$

```
  a --
      |
      |-- (multiply)--
      |               |
  x --                |-- (add) --> output
                      |
                  b --
```

In [ ]:
# Initialize variables and input data
x_data = tf.keras.Input(dtype=tf.float32, shape=(1,))
x_val = 5.
a = tf.Variable(1., dtype=tf.float32)
b = tf.Variable(1., dtype=tf.float32)

# Add a layer which computes f(x) = a * x
multiply_layer = tf.keras.layers.Lambda(lambda x:tf.multiply(a, x))

# Add a layer which computes f(x) = b + x
add_layer = tf.keras.layers.Lambda(lambda x:tf.add(b, x))

res = multiply_layer(x_data)
outputs = add_layer(res)

# Build the model
model = tf.keras.Model(inputs=x_data, outputs=outputs, name="gate_2")

print(model.summary())

# Optimizer
optimizer=tf.keras.optimizers.SGD(0.01)

# Run loop across gate
print('Optimizing two Gate Output to 50.')
for i in range(10):
    
    # Open a GradientTape.
    with tf.GradientTape(persistent=True) as tape:
        
        # Forward pass.
        two_gate_output = model(x_val)
        
        # Loss value as the difference between
        # the output and a target value, 50.
        loss_value = tf.square(tf.subtract(two_gate_output, 50.))
        
    # Get gradients of loss with reference to the variables "a" and "b" to adjust.
    gradients_a = tape.gradient(loss_value, a)
    gradients_b = tape.gradient(loss_value , b)
    
    # Update the variables "a" and "b" of the model.
    optimizer.apply_gradients(zip([gradients_a, gradients_b], [a, b]))
    
    print("Step: {} ==> {} * {} + {}= {}".format(i, a.numpy(),
                                                 x_val, b.numpy(),
                                                 a.numpy() * x_val + b.numpy()))
    
    
    

The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=() dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (lambda_2), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=() dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "gate_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
________________________________________________________________